In [1]:
import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

from soma import aims
import numpy as np
import pandas as pd
import os
import glob

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
a = ana.Anatomist()

existing QApplication: 0
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-jc225751'


create qapp
done
Starting Anatomist.....
config file : /casa/home/.anatomist/config/settings.cfg
PyAnatomist Module present
PythonLauncher::runModules()
global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
loading module meshsplit
loading module palettecontrols
loading module volumepalettes
loading module save_resampled
loading module profilewindow
loading module statsplotwindow
loading module anacontrolmenu
loading module infowindow
loading module foldsplit
loading module simple_controls
loading module histogram
loading module valuesplotwindow
loading module ana_image_math
loading module modelGraphs
loading module paletteViewer
loading module bsa_proba
loading module gradientpalette
loading module selection
loading module gltf_io
all python modules loaded
Anatomist started.


In [3]:
path = "/neurospin/dico/data/deep_folding/current/models/Champollion_V0_trained_on_UKB40/SC-sylv_right"
lab = "gravityCenter_y"
model_path = glob.glob(f"{path}/*")[0]
embeddings_file = f"{model_path}/hcp_random_epoch80_embeddings/full_embeddings.csv"
participants_file = "/neurospin/dico/data/human/hcp/derivatives/morphologist-2023/morphometry/spam_recognition/morpho_talairach/morpho_S.C._right.dat"

In [4]:
participants = pd.read_csv(participants_file, sep=' ', index_col=0)
participants = participants[[lab]].dropna()
participants

,gravityCenter_y
subject,
100206,23.6737
100307,17.6998
100408,20.6022
100610,20.0265
101006,18.7678
...,...
992774,18.2473
993675,18.7725
994273,19.0850


In [5]:
ukb_emb = pd.read_csv(f"{embeddings_file}", index_col=0)
merged = participants[[lab]].merge(ukb_emb, left_index=True, right_index=True)
label = merged.iloc[:,0:1]
label.head()

,gravityCenter_y
subject,
100206,23.6737
100307,17.6998
100408,20.6022
100610,20.0265
101006,18.7678


### Variable definitions

In [6]:
side = "R" # "R" or "L"
region = "S.C.-sylv." # "ORBITAL", "CINGULATE", "SC-sylv", "F.I.P."
database='hcp'

In [7]:
# %matplotlib inline
# df[["p"]].plot.hist()
# plt.show()

### Function definitions

In [8]:
def to_bucket(obj):
    if obj.type() == obj.BUCKET:
        return obj
    avol = a.toAimsObject(obj)
    c = aims.Converter(intype=avol, outtype=aims.BucketMap_VOID)
    abck = c(avol)
    bck = a.toAObject(abck)
    bck.releaseAppRef()
    return bck

def build_gradient(pal):
    gw = ana.cpp.GradientWidget(None, 'gradientwidget', pal.header()['palette_gradients'])
    gw.setHasAlpha(True)
    nc = pal.shape[0]
    rgbp = gw.fillGradient(nc, True)
    rgb = rgbp.data()
    npal = pal.np['v']
    pb = np.frombuffer(rgb, dtype=np.uint8).reshape((nc, 4))
    npal[:, 0, 0, 0, :] = pb
    npal[:, 0, 0, 0, :3] = npal[:, 0, 0, 0, :3][:, ::-1]  # BGRA -> RGBA
    pal.update()

def buckets_average(subject_id_list, dataset_name_list):
    dic_vol = {}
    dim = 0
    rep = 0
    if len(subject_id_list) == 0:
        return False
    while dim == 0 and rep < len(subject_id_list):
        if dataset_name_list[rep].lower() in ['ukb', 'ukbiobank', 'projected_ukb']:
            dataset = 'UkBioBank'
        elif dataset_name_list[rep].lower() in ['hcp']:
            dataset = 'hcp'
        mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops"
        if os.path.isfile(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz'):
            sum_vol = aims.read(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz').astype(float)
            dim = sum_vol.shape
            sum_vol.fill(0)
        else: 
            print(f'FileNotFound {mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz')
            #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id_list[0]}_cropped_skeleton.nii.gz')
        rep += 1

    for subject_id, dataset in zip(subject_id_list,dataset_name_list):
        if dataset.lower() in ['ukb', 'ukbiobank',  'projected_ukb']:
            dataset = 'UkBioBank'
        elif dataset.lower() == 'hcp':
            dataset = 'hcp'
            
        mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops"

        if os.path. isfile(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz'):
            vol = aims.read(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
            # compare the dim with the first file dim

            if vol.np.shape != dim:
                raise ValueError(f"{subject_id_list[0]} and {subject_id} must have the same dim")

                
            # to have a binary 3D structure
            dic_vol[subject_id] = (vol.np > 0).astype(int)
            sum_vol.np[:] += (vol.np > 0).astype(int) 
        else: 
            print(f'FileNotFound {mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
            #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')

    sum_vol = sum_vol / len(subject_id_list)
    print(sum_vol, sum_vol.shape)
    return sum_vol

In [9]:
def visualize_averages_along_parameter(df, column_name, database, nb_subjects_per_average=200, nb_columns=3, nb_lines=1):
    # anatomist objects
    global _block
    global _average_dic
    nb_windows = nb_columns * nb_lines
    _average_dic = {}
    step = nb_subjects_per_average # number of subjects on which average is done
    # Creates the block if it has not been created
    try:
        _block
    except NameError:
        _block = a.createWindowsBlock(nb_columns)

    # Order according to column_name
    df = df.sort_values(column_name, ascending=False)
    predict_proba = df[[column_name]]

    # Creates dictionary of subjects to average
    dic_packages = {}
    for i in range(0,len(predict_proba),step):
        list_idx = (predict_proba.index[i:i+step].to_numpy())
        dic_packages[i//step] = list_idx
    list_database = [database for i in range(step)]
    n_pack = len(dic_packages)

    # Loop of display averages
    list_pack = [int(np.ceil(i*n_pack/float(nb_windows))) for i in range(0,nb_windows)]
    sum_vol_list = []
    for i in list_pack:
        sum_vol_list.append(buckets_average(dic_packages[i], list_database))
        _average_dic[f'a_sum_vol{i}'] = a.toAObject(sum_vol_list[-1])
        _average_dic[f'a_sum_vol{i}'].setPalette(minVal=0, absoluteMode=True)
        #wsum = a.createWindow('Sagittal', block=block)
        #wsum.addObjects(a_sum_vol)
        _average_dic[f'rvol{i}'] = a.fusionObjects(
            objects=[_average_dic[f'a_sum_vol{i}']],
            method='VolumeRenderingFusionMethod')
        _average_dic[f'rvol{i}'].releaseAppRef()
        # custom palette
        n = len(dic_packages[i])
        pal = a.createPalette('VR-palette')
        pal.header()['palette_gradients'] = '0;0.459574;0.497872;0.910638;1;1#0;0;0.52766;0.417021;1;1#0;0.7;1;0#0;0;0.0297872;0.00851064;0.72766;0.178723;0.957447;0.808511;1;1'
        #f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.5/n};0;1;1'
        build_gradient(pal)
        _average_dic[f'rvol{i}'].setPalette('VR-palette', minVal=0.05, absoluteMode=True)
        pal2 = a.createPalette('slice-palette')
        pal2.header()['palette_gradients'] = '0;0.459574;0.497872;0.910638;1;1#0;0;0.52766;0.417021;1;1#0;0.7;1;0#0;0;0.0297872;0.00851064;0.72766;0.178723;0.957447;0.808511;1;1'
        #f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.3/n};0;{0.7/n};1;1;1'
        build_gradient(pal2)
        _average_dic[f'a_sum_vol{i}'].setPalette('slice-palette')
        # rvol.palette().fill()
        _average_dic[f'wvr{i}'] = a.createWindow('3D', block=_block)
        _average_dic[f'wvr{i}'].addObjects(_average_dic[f'rvol{i}'])
        
    return sum_vol_list

### Visualization

In [10]:
# block = a.createWindowsBlock(10)
sum_vol_list = visualize_averages_along_parameter(label, lab, database)

<soma.aims.Volume_DOUBLE object at 0x7478300879a0> (42, 34, 49, 1)
Multitexturing present
function glActiveTexture found.
function glClientActiveTexture found.
function glBlendEquation found.
function glTexImage3D found.
function glMultiTexCoord3f found.
function glBindFramebuffer found.
function glBindRenderbuffer found.
function glFramebufferTexture2D found.
function glGenFramebuffers found.
function glGenRenderbuffers found.
function glFramebufferRenderbuffer found.
function glRenderbufferStorage found.
function glCheckFramebufferStatus found.
function glDeleteRenderbuffers found.
function glDeleteFramebuffers found.
Number of texture units: 4
function glUniform1f found.
function glUniform1i found.
function glUniform4fv found.
function glGetUniformLocation found.
function glMultTransposeMatrixf found.
function glAttachShader found.
function glDetachShader found.
function glCompileShader found.
function glCreateProgram found.
function glCreateShader found.
function glDeleteProgram fo

In [11]:
type(_block)

anatomist.direct.api.Anatomist.AWindowsBlock

In [12]:
_average_dic

{'a_sum_vol0': <anatomist.cpp.anatomist.SliceableObject object at 0x7479003bcd30>,
 'rvol0': <anatomist.cpp.anatomist.MObject object at 0x747828f9c1f0>,
 'wvr0': <anatomist.cpp.weak_shared_ptr_AWindow object at 0x747824708820>,
 'a_sum_vol2': <anatomist.cpp.anatomist.SliceableObject object at 0x7478251dda20>,
 'rvol2': <anatomist.cpp.anatomist.MObject object at 0x7478251dd870>,
 'wvr2': <anatomist.cpp.weak_shared_ptr_AWindow object at 0x7478251dd900>,
 'a_sum_vol4': <anatomist.cpp.anatomist.SliceableObject object at 0x747824724430>,
 'rvol4': <anatomist.cpp.anatomist.MObject object at 0x747824724160>,
 'wvr4': <anatomist.cpp.weak_shared_ptr_AWindow object at 0x7478247244c0>}

In [13]:
vol_diff = sum_vol_list[-1] - sum_vol_list[0]
a_diff = a.toAObject(vol_diff)
a_diff.setPalette(minVal=0, absoluteMode=True)
fusion_diff = a.fusionObjects(
        objects=[a_diff],
        method='VolumeRenderingFusionMethod')
fusion_diff.releaseAppRef()
fusion_diff.setPalette('VR-palette', minVal=0.05, absoluteMode=True)
a_diff.setPalette('slice-palette')
win_diff = a.createWindow('3D', block=_block)
win_diff.addObjects(fusion_diff)